In [1]:
import os

os.environ["WF_TRACE_SERVER_URL"] = "http://127.0.01:6345"

In [2]:
import weave

In [3]:
client = weave.init("remote_model_demo_4")

Logged in as Weights & Biases user: timssweeney.
View Weave data at https://wandb.ai/timssweeney/remote_model_demo_4/weave


In [4]:
# Demonstrates creating a model in python

from weave.builtin_objects.models.CompletionModel import LiteLLMCompletionModel

model = LiteLLMCompletionModel(
    model="gpt-4o",
    messages_template=[
        {
            "role": "system",
            "content": "Please extract the name and age from the following text",
        },
        {"role": "user", "content": "{user_input}"},
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "Person",
            "schema": {
                "type": "object",
                "properties": {
                    "age": {"type": "integer"},
                    "name": {"type": "string"},
                },
            },
        },
    },
)

res = model.predict(user_input="Hello, my name is Fred and I am 30 years old.")

print(model.ref.uri())

print(res)

🍩 https://wandb.ai/timssweeney/remote_model_demo_4/r/call/0193b4ad-e5f5-7c10-a41f-ee06bc0c34ea
weave:///timssweeney/remote_model_demo_4/object/LiteLLMCompletionModel:KBsfUswVpEHFYmZuJjmhM2YH4EttkRZJSoH0Z0ZaNRY
{'name': 'Fred', 'age': 30}


In [5]:
# Demonstrates calling a model created in python

from weave.trace_server.trace_server_interface import CallMethodReq

call_res = client.server.call_method(
    CallMethodReq(
        project_id=client._project_id(),
        object_ref=model.ref.uri(),
        method_name="predict",
        args={"user_input": "Hello, my name is Charles and I am 40 years old."},
    )
)

call_res

CallMethodRes(call_id='0193b4af-f064-7f92-8852-ed3024344dac', output={'name': 'Charles', 'age': 40})

In [7]:
# Demonstrates creating a model in the UI - notice the digest match

from weave.trace_server.trace_server_interface import ObjCreateReq

obj_res = client.server.obj_create(
    ObjCreateReq.model_validate(
        {
            "obj": {
                "project_id": client._project_id(),
                "object_id": "LiteLLMCompletionModel",
                "val": {
                    "model": "gpt-4o",
                    "messages_template": [
                        {
                            "role": "system",
                            "content": "Please extract the name and age from the following text!",
                        },
                        {"role": "user", "content": "{user_input}"},
                    ],
                    "response_format": {
                        "type": "json_schema",
                        "json_schema": {
                            "name": "Person",
                            "schema": {
                                "type": "object",
                                "properties": {
                                    "age": {"type": "integer"},
                                    "name": {"type": "string"},
                                },
                            },
                        },
                    },
                },
                "set_leaf_object_class": "LiteLLMCompletionModel",
            }
        }
    )
)

obj_res

ObjCreateRes(digest='k85wXnWLVxpHujpohAqNBIirXZSM6XRSOSk84n1XR84')

In [8]:
# Demonstrates fetching a model in python that was created in the UI

from weave.trace.refs import ObjectRef

gotten_model = weave.ref(
    ObjectRef(
        entity=client.entity,
        project=client.project,
        name="LiteLLMCompletionModel",
        _digest=obj_res.digest,
    ).uri()
).get()

res = gotten_model.predict(user_input="Hello, my name is Fred and I am 30 years old.")
res

🍩 https://wandb.ai/timssweeney/remote_model_demo_4/r/call/0193b4b2-897b-77e0-b404-236415a689b9


{'name': 'Fred', 'age': 30}

In [9]:
gotten_model

LiteLLMCompletionModel(name=None, description=None, model='gpt-4o', messages_template=WeaveList([{'role': 'system', 'content': 'Please extract the name and age from the following text!'}, {'role': 'user', 'content': '{user_input}'}]), response_format=WeaveDict({'type': 'json_schema', 'json_schema': {'name': 'Person', 'schema': {'type': 'object', 'properties': {'age': {'type': 'integer'}, 'name': {'type': 'string'}}}}}))